In [9]:
import pandas as pd

In [10]:
df = pd.read_csv('data_final.csv')

In [11]:
df.shape

(37790, 7)

In [12]:
df.head()

,ID commande,Payment date,Nom du fabricant,Product Category Default,Article,Code postal (Facture),Ville (Facture)
0,1743,08.01.2018,FERME BIXARTEA,CRÈMERIE,breuil (375g).,64250.0,SAINT PALAIS
1,1743,08.01.2018,FERME BIXARTEA,FROMAGES,fromage de brebis fermier (650g environ) +6 mo...,64250.0,SAINT PALAIS
2,1743,08.01.2018,FERME AROZTEGIA,FRUITS & LÉGUMES,radis noir (la botte),64250.0,SAINT PALAIS
3,1743,08.01.2018,FERME ESNEKIAK,CRÈMERIE,Yaourt nature bio au lait entier (4 x125 gr).,64250.0,SAINT PALAIS
4,1743,08.01.2018,FERME BELAZKABIETA,ÉPICERIE,BIZIGAI Piment d'Espelette AOP (50 gr),64250.0,SAINT PALAIS


## 1 Modification du nom des colonnes

In [13]:
df.columns

Index(['ID commande', 'Payment date', 'Nom du fabricant',
       'Product Category Default', 'Article', 'Code postal (Facture)',
       'Ville (Facture)'],
      dtype='object')

In [14]:
df.columns = ['commande','date','producteur','categorie','nom_produit','code_postal','commune']

## 2 Modification contenu code_postal

In [19]:
df.code_postal.dtypes

dtype('float64')

In [22]:
df.code_postal = df.code_postal.astype(int)

In [23]:
df.dtypes

commande        int64
date           object
producteur     object
categorie      object
nom_produit    object
code_postal     int64
commune        object
dtype: object

In [24]:
df.head()

,commande,date,producteur,categorie,nom_produit,code_postal,commune
0,1743,08.01.2018,FERME BIXARTEA,CRÈMERIE,breuil (375g).,64250,SAINT PALAIS
1,1743,08.01.2018,FERME BIXARTEA,FROMAGES,fromage de brebis fermier (650g environ) +6 mo...,64250,SAINT PALAIS
2,1743,08.01.2018,FERME AROZTEGIA,FRUITS & LÉGUMES,radis noir (la botte),64250,SAINT PALAIS
3,1743,08.01.2018,FERME ESNEKIAK,CRÈMERIE,Yaourt nature bio au lait entier (4 x125 gr).,64250,SAINT PALAIS
4,1743,08.01.2018,FERME BELAZKABIETA,ÉPICERIE,BIZIGAI Piment d'Espelette AOP (50 gr),64250,SAINT PALAIS


In [25]:
df.code_postal.unique()

array([64250, 64240, 64780, 64480, 64640, 64100, 64990, 64600, 64310,
       64210, 64200, 64120, 40220, 64122, 40440, 77700, 64340, 35580,
       40530, 31300, 24270, 64500, 38000, 64330, 64190, 64140, 64220,
       79400, 40300, 64300, 78000, 64259, 40390])